In [1]:
from pymongo import MongoClient
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize
from gensim import corpora
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel

import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import re
import pandas as pd
import numpy as np
import gensim
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import emoji
import pickle
import string


/home/tsbh/anaconda3/lib/python3.9/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/home/tsbh/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [2]:
df=pd.read_excel("data_training_lexicon1.xlsx",0)
df

,Unnamed: 0,Cleaned_Text
0,0,ngeri dar awal kpu bukan wasit yang baik pante...
1,1,rkuhp buat nyali kurang
2,2,pasalpasal rkuhp lebih bahaya belum syariahmew...
3,3,UU omnibuslaw saja sahkan apa rkuhp
4,4,enggan perintah buka draf baru rkuhp nilai tun...
...,...,...
4996,4996,laku bom bunuh diri polsek astana anyar bawa b...
4997,4997,benar beberapa rkuhp ganti ganti sih
4998,4998,lord rangga tinggal tepat sah nya rkuhp apa ma...
4999,4999,list jadi minggu awal desember gempa bom bunuh...


In [3]:
df=pd.DataFrame(df)
df

,Unnamed: 0,Cleaned_Text
0,0,ngeri dar awal kpu bukan wasit yang baik pante...
1,1,rkuhp buat nyali kurang
2,2,pasalpasal rkuhp lebih bahaya belum syariahmew...
3,3,UU omnibuslaw saja sahkan apa rkuhp
4,4,enggan perintah buka draf baru rkuhp nilai tun...
...,...,...
4996,4996,laku bom bunuh diri polsek astana anyar bawa b...
4997,4997,benar beberapa rkuhp ganti ganti sih
4998,4998,lord rangga tinggal tepat sah nya rkuhp apa ma...
4999,4999,list jadi minggu awal desember gempa bom bunuh...


In [4]:
df=df[['Cleaned_Text']]
df

,Cleaned_Text
0,ngeri dar awal kpu bukan wasit yang baik pante...
1,rkuhp buat nyali kurang
2,pasalpasal rkuhp lebih bahaya belum syariahmew...
3,UU omnibuslaw saja sahkan apa rkuhp
4,enggan perintah buka draf baru rkuhp nilai tun...
...,...
4996,laku bom bunuh diri polsek astana anyar bawa b...
4997,benar beberapa rkuhp ganti ganti sih
4998,lord rangga tinggal tepat sah nya rkuhp apa ma...
4999,list jadi minggu awal desember gempa bom bunuh...


In [5]:
df = df.rename(columns={"Cleaned_Text": "caption"})
df

,caption
0,ngeri dar awal kpu bukan wasit yang baik pante...
1,rkuhp buat nyali kurang
2,pasalpasal rkuhp lebih bahaya belum syariahmew...
3,UU omnibuslaw saja sahkan apa rkuhp
4,enggan perintah buka draf baru rkuhp nilai tun...
...,...
4996,laku bom bunuh diri polsek astana anyar bawa b...
4997,benar beberapa rkuhp ganti ganti sih
4998,lord rangga tinggal tepat sah nya rkuhp apa ma...
4999,list jadi minggu awal desember gempa bom bunuh...


In [6]:
df

,caption
0,ngeri dar awal kpu bukan wasit yang baik pante...
1,rkuhp buat nyali kurang
2,pasalpasal rkuhp lebih bahaya belum syariahmew...
3,UU omnibuslaw saja sahkan apa rkuhp
4,enggan perintah buka draf baru rkuhp nilai tun...
...,...
4996,laku bom bunuh diri polsek astana anyar bawa b...
4997,benar beberapa rkuhp ganti ganti sih
4998,lord rangga tinggal tepat sah nya rkuhp apa ma...
4999,list jadi minggu awal desember gempa bom bunuh...


In [7]:
df['caption'].replace('', np.nan, inplace=True)

In [8]:
df.dropna(subset=['caption'], inplace=True)

In [9]:
df

,caption
0,ngeri dar awal kpu bukan wasit yang baik pante...
1,rkuhp buat nyali kurang
2,pasalpasal rkuhp lebih bahaya belum syariahmew...
3,UU omnibuslaw saja sahkan apa rkuhp
4,enggan perintah buka draf baru rkuhp nilai tun...
...,...
4996,laku bom bunuh diri polsek astana anyar bawa b...
4997,benar beberapa rkuhp ganti ganti sih
4998,lord rangga tinggal tepat sah nya rkuhp apa ma...
4999,list jadi minggu awal desember gempa bom bunuh...


In [10]:
tknzr = TweetTokenizer()

def tokenize(text):
    lda_tokens = []
    tokens = tknzr.tokenize(text)
    for token in tokens:
        lda_tokens.append(token)
    return lda_tokens

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    return tokens

In [11]:
for data in df["caption"]:
    tokens = prepare_text_for_lda(data)
    print(tokens)

['ngeri', 'dar', 'awal', 'kpu', 'bukan', 'wasit', 'yang', 'baik', 'pantesan', 'buru', 'gol', 'rkuhp', 'presiden', 'yang', 'lahir', 'curang', 'aman']
['rkuhp', 'buat', 'nyali', 'kurang']
['pasalpasal', 'rkuhp', 'lebih', 'bahaya', 'belum', 'syariahmewujudkanpersatuan']
['UU', 'omnibuslaw', 'saja', 'sahkan', 'apa', 'rkuhp']
['enggan', 'perintah', 'buka', 'draf', 'baru', 'rkuhp', 'nilai', 'tunjuk', 'bbc']
['nah', 'kamu', 'yang', 'benar', 'saja', 'sudah', 'fitnah', 'jatohnya', 'terkena', 'pasal', 'rkuhp', 'baru', 'hina', 'presiden', 'tambah', 'tindak', 'pidana', 'terorisme', 'jelas', 'dukung', 'gera', 'terorisme', 'orochi', 'm', 'top']
['maling', 'bintang', 'emon', 'vs', 'mamat', 'alkatiri', 'resiko', 'kritik', 'perintah', 'rkuhp', 'via']
['rkuhp']
['harus', 'sih', 'sadar', 'ta', 'tapi', 'ntah', 'lh', 'mungkin', 'belum', 'baca', 'rkuhp', 'yang', 'baru', 'ganggu', 'tetangga']
['salut', 'pasang', 'connect', 'zenly', 'larang', 'nongkrong', 'lawan', 'jenis', 'saling', 'tukar', 'password', 'ig',

['ancam', 'bebas', 'sipil', 'bebas', 'ekspres', 'bahas', 'terkena', 'polemik', 'UU', 'kuhp', 'sebut', 'suara', 'bebas', 'selenggara', 'diskusi', 'webinar', 'forum', 'bebas', 'angkat', 'topik', 'tantang', 'ham', 'pasca', 'kesah', 'rkuhp']
['undang', 'diskusi', 'webinar', 'forum', 'bebas', 'tantang', 'ham', 'pasca', 'pengesahaan', 'rkuhp']
['orang', 'aman', 'polisi', 'usai', 'demo', 'ricuh', 'tolak', 'rkuhp', 'bandung']
['rkuhp', 'baik', 'sistem', 'hukum', 'baik', 'rakyat', 'indonesia']
['rkuhp', 'baru', 'sudah', 'sangat', 'ideal', 'ganti', 'kuhp', 'zaman', 'kolonial', 'belandarkuhp', 'baru', 'sudah', 'sangat', 'ideal', 'ganti', 'kuhp', 'zaman', 'kolonial', 'belanda']
['rkuhp', 'jamin', 'pasti', 'hukum', 'jadi', 'kuhp', 'tanah', 'air', 'makin', 'orientasi', 'hukum', 'pidana', 'modern']
['rkuhp', 'mantap']
['baru', 'rkuhp', 'rupa', 'bentuk', 'hormat', 'hukum', 'laku', 'masyarakat']
['rkuhp', 'buat', 'profesional', 'kredibel']
['rkuhp', 'sesuai', 'hukum', 'pidana', 'modern', 'universal', '

['tempo', 'tulis', 'usia', 'citacita', 'reformasi', 'tumbang', 'orde', 'baru', 'tahun', 'rkuhp']
['klaster', 'aktivis', 'puisi', 'rkuhp', 'thank', 'you', 'selebriti', 'pengendorse', 'rkuhp', 'kalian', 'senang', 'kan', 'hari', 'UU', 'cipta', 'kerja', 'minerba', 'kpk', 'sehat', 'ikn', 'kini', 'rkuhp', 'semua', 'sulut', 'masalah', 'UU', 'polisi', 'biar']
['klaster', 'pro', 'oposisi', 'pks', 'tolak', 'pasal', 'hina', 'presiden', 'rkuhp', 'ylbhi', 'rkuhp', 'potensi', 'jerat', 'kelompok', 'seru', 'syariah', 'islam', 'paripurna', 'tadi', 'saya', 'interupsi', 'tidak', 'tuju', 'pasal', 'rkuhp', 'yang', 'pidana', 'siapa', 'yang', 'hina', 'pres']
['mayoritas', 'tolak', 'rkuhp', 'tampak', 'satu', 'klaster', 'besar', 'warna', 'merah', 'tanda', 'sentimen', 'negatif', 'rkuhp', 'beberapa', 'top', 'influencer', 'klaster', 'kontra']
['bicara', 'nada', 'positif', 'rkuhp', 'sah', 'jadi', 'kuhp', 'sosialisasi', 'kuhp', 'tanggung', 'jawab', 'semua', 'pihak', 'umum', 'publik', 'nilai', 'rkuhp', 'masalah', 'b

['dukung', 'kesah', 'rkuhp', 'tatanan', 'hukum', 'lebih', 'baik']
['dukung', 'kesah', 'rkuhp', 'tatanan', 'hukum', 'lebih', 'baik']
['rkuhp', 'sah', 'uus', 'tertawa', 'deddy', 'corbuzier', 'soal', 'pasal', 'bebas', 'dapat', 'news', 'berita']
['rkuhp', 'sengaja', 'gulir', 'kasus', 'sambo', 'plin', 'plan', 'fraksi', 'koalisi', 'sengaja', 'benar', 'omg']
['rkuhp', 'sempurna', 'tata', 'regulasi', 'hukum', 'jokowipresidenku', 'pembangunanuntukrakyat', 'presidenjokowihebat', 'jokowimembangunnegeri', 'jokowimajubersamaindonesia', 'jokowibapakinfrastruktur']
['hahahahahahaha', 'ngecek', 'notes', 'handphone', 'terus', 'nemu', 'tulis', 'inspirasi', 'rkuhp', 'waktu', 'sah', 'penis', 'terlalu', 'kecil', 'buah', 'bui', 'satya', 'karasuma', 'selamat', 'baca']
['gas', 'terap', 'rkuhp', 'ayat', 'bagi', 'empat', 'suami', 'istri', 'lapor', 'kalau', 'selingkuh']
['ngotot', 'rkuhp', 'sah', 'fenomena', 'asingo', 'sempil']
['cewek', 'aktivis', 'tutup', 'mulut', 'tanda', 'tolak', 'rkuhp', 'kan']
['breaking',

['kesah', 'ruu', 'kuhp', 'sangat', 'penting', 'legacy', 'waris', 'bangsa', 'penting', 'ujar', 'benny', 'rkuhp', 'hukumindonesia', 'cr', 'welcome', 'to', 'jakarta']
['rkuhp', 'yang', 'sah', 'kan', 'pmrnth', 'tidk', 'bakal', 'pengaruh', 'pariwisata', 'endonesia']
['rkuhp', 'sah', 'kan', 'pemrintah', 'tak', 'bkl', 'pengaruh', 'pariwisata', 'indonesia']
['rkuhp', 'sah', 'kan', 'perintah', 'pengaruh', 'pariwisata', 'indonesia']
['rkuhp', 'yang', 'sah', 'kan', 'pmrnth', 'tidk', 'bakal', 'pengaruh', 'pariwisata', 'endonesia']
['rkuhp', 'sah', 'kan', 'perintah', 'pengaruh', 'pariwisata', 'indonesia']
['rkuhp', 'pasal', 'tangkap', 'orang', 'seperti']
['pasca', 'rkuhp', 'sahkan']
['bagus', 'hukum', 'modern', 'indonesia', 'negara', 'intervensi', 'rkuhp', 'savekpk', 'berantaskorupsi', 'hukumdankeadilan', 'indonesiabebaskorupsi']
['bagus', 'hukum', 'modern', 'indonesia', 'negara', 'intervensi', 'rkuhp', 'savekpk', 'berantaskorupsi', 'hukumdankeadilan', 'indonesiabebaskorupsi']
['bagus', 'hukum', 'm

...

['menteri', 'hukum', 'hak', 'asasi', 'manusia', 'menkumham', 'yasonna', 'h', 'laoly', 'kata', 'masyarakat', 'tuju', 'rkuhp', 'aju', 'gugat', 'mahkamah', 'konstitusi', 'mk', 'jadi', 'undangundang', 'menkumham', 'uukuhp', 'kuhp', 'rkuhp', 'yasonnalaoly', 'dprri', 'pdiperjuangan']
['pasal', 'colek', 'lgbt', 'rkuhp', 'sadar', 'kalau', 'exstrimis', 'makin', 'tingkat']
['love', 'u', 'more', 'than', 'jumlah', 'pasal', 'ugalugalan', 'rkuhp', 'ana']
['kesah', 'rkuhp', 'resah', 'kalang', 'pers', 'smsi', 'bakal', 'gugat', 'lalu', 'mk']
['skripsi', 'pidana', 'makin', 'seru', 'kali', 'now', 'that', 'rkuhp', 'sah']
['rkuhp', 'sekarang', 'produk', 'hukum', 'modern', 'tahan', 'butuh', 'acu', 'hukum', 'tegas', 'tak', 'buat', 'ragu', 'aparat', 'negara', 'tindak']
['urgensi', 'kesah', 'rkuhp', 'semata', 'penting', 'maysarkat', 'indonesia']
['masyarakat', 'tetap', 'salur', 'aspirasi', 'ada', 'rkuhp', 'baru', 'sah', 'perintah']
['ken', 'kaget', 'cara', 'kaya', 'gin', 'pernah', 'pakai', 'waktu', 'kesah', 'o

['sharing', 'ko', 'potong', 'sharing', 'ko', 'tidak', 'utuh', 'rkuhp', 'sempurna', 'yang', 'puas', 'sampai', 'jelas', 'pasal', 'yang', 'debat', 'jangan', 'asal', 'tolak', 'seluruh', 'lebih', 'bagus', 'banding', 'dg', 'kuhp', 'lama', 'yuriprudensinya']
['pasal', 'rkuhp']
['rkuhp', 'sah', 'kantor', 'tri', 'hukum', 'ham', 'bakar']
['jubir', 'sosialisasi', 'rkuhp', 'sebut', 'selebgram', 'ikut', 'endorse', 'rkuhp', 'hukum', 'lanngar']
['tuai', 'kontroversi', 'kesah', 'rkuhp', 'tak', 'bebas', 'penting', 'kuasa', 'serbaserbi', 'mmc', 'lengkap', 'video', 'ikut', 'jangan', 'lupa', 'share', 'seluasluasnya', 'kesah', 'rkuhp', 'kontroversi', 'serbaserbi', 'mmc', 'islam', 'kaffah']
['potensi', 'turun', 'capai', 'sehat', 'seksual', 'reproduksi', 'indonesia', 'cukup', 'rendah', 'diah', 'saminarsih', 'cisdi', 'undangundang', 'problematik', 'negara', 'problematik', 'kuhpbuatanindonesia', 'kuhptanpapasalkaret', 'rkuhp']
['aduh', 'rkuhp', 'sudah', 'sahkanehm', 'kok', 'gesa', 'gesa', 'sihada', 'apa', 'ger

['kesah', 'rkuhp', 'gass', 'saja', 'padahal', 'pekan', 'masyarakat', 'tengah', 'langsung', 'aksi', 'tolak', 'islamkaffah', 'serbaserbi', 'mmc']
['tengah', 'deras', 'arus', 'tolak', 'beberapa', 'pasal', 'rkuhp', 'dpr', 'ri', 'perintah', 'justru', 'kesah', 'jadi', 'undangundang', 'rapat', 'paripurna', 'gelar', 'kompleks', 'parlemen', 'selasa', 'kesah', 'rkuhp', 'islamkaffah', 'serbaserbi', 'mmc']
['tengah', 'deras', 'arus', 'tolak', 'beberapa', 'pasal', 'rkuhp', 'dpr', 'ri', 'perintah', 'justru', 'kesah', 'jadi', 'undangundang', 'rapat', 'paripurna', 'gelar', 'kompleks', 'parlemen', 'selasa', 'kesah', 'rkuhp', 'islamkaffah', 'serbaserbi', 'mmc']
['tengah', 'deras', 'arus', 'tolak', 'beberapa', 'pasal', 'rkuhp', 'dpr', 'ri', 'perintah', 'justru', 'kesah', 'jadi', 'undangundang', 'rapat', 'paripurna', 'gelar', 'kompleks', 'parlemen', 'selasa', 'kesah', 'rkuhp', 'islamkaffah', 'serbaserbi', 'mmc']
['kesah', 'rkuhp', 'tuai', 'kontroversi', 'pasal', 'masyarakat', 'banyak', 'laku', 'tolak', 'l

['tolak', 'rkuhp']
['tengah', 'deras', 'arus', 'tolak', 'beberapa', 'pasal', 'rancang', 'kitab', 'undangundang', 'hukum', 'pidana', 'rkuhp', 'dpr', 'ri', 'perintah', 'justru', 'kesah', 'jadi', 'undangundang', 'simak', 'lengkap', 'serba', 'serbi', 'mmc', 'islamkaffah']
['min', 'bagaimana', 'tanggap', 'rkuhp']
['kesah', 'rkuhp', 'banyak', 'bicara', 'karena', 'tak', 'tepatbsasaran', 'yuk', 'simak', 'lengkap', 'islamkaffah']
['fix', 'periode', 'tahun', 'depan', 'ngemeng', 'sedikit', 'bakal', 'terkena', 'UU', 'pasal', 'rkuhp']
['gila', 'keren', 'banget', 'dinasti', 'politik', 'rakyat', 'bakal', 'tindas', 'rkuhp', 'rip', 'demokrasi', 'indonesia', 'semua', 'hrs', 'satu', 'gugat', 'rkuhp']
['kesah', 'rkuhp', 'tuai', 'banyakkk', 'dapat', 'tepat', 'yuk', 'simak', 'lengkap', 'islamkaffah']
['tuai', 'kontroversi', 'kesah', 'rkuhp', 'tak', 'bebas', 'penting', 'kuasa', 'lengkap', 'serbaserbi', 'mmc', 'islamkaffah']
['kesah', 'rkuhp', 'benar', 'pihak', 'siapa', 'yuk', 'simak', 'lengkap', 'islamkaffah

['rebut', 'rejeki', 'orang', 'penjara', 'rkuhp', 'iki', 'tulung', 'disahke', 'muehehehe']
['yasonna', 'jelas', 'dapat', 'pidana', 'yang', 'atur', 'rkuhp', 'pidana', 'pokok', 'pidana', 'tambah', 'pidana', 'yang', 'sifat', 'khusus', 'yasonasahkanrkuhp']
['kesah', 'rkuhp', 'lalu', 'sosialisasi', 'cukup', 'panjang', 'minta', 'masuk', 'ahli', 'yasonasahkanrkuhp']
['lihat', 'banyak', 'orang', 'public', 'figure', 'mayah', 'mayah', 'rkuhp', 'sah', 'yakin', 'mreka', 'sangat', 'benci', 'orang', 'hukum', 'almamater', 'babi', 'buta', 'hahahahahaha']
['rkuhp', 'bawa', 'misi', 'dekolonialisasi', 'konsolidasi', 'harmonisasi', 'hukum', 'pidana', 'yasonasahkanrkuhp']
['perintah', 'kesah', 'rkuhp', 'jadi', 'undangundang', 'reformasi', 'hukum', 'sesuai', 'tuju', 'bangun', 'nasional', 'wujud', 'cipta', 'masyarakat', 'adil', 'makmur', 'yasonasahkanrkuhp']
['selang', 'rkuhp', 'sah', 'political', 'show', 'full', 'via']
['bukan', 'cuma', 'sih', 'mas', 'fokus', 'orang', 'jadi', 'geser', 'jadi', 'bukan', 'rkuhp

In [12]:
# import random
text_data = []
for line in df["caption"]:
    tokens = prepare_text_for_lda(line)
#     if random.random() > .99:
    print(tokens)
    text_data.append(tokens)

['ngeri', 'dar', 'awal', 'kpu', 'bukan', 'wasit', 'yang', 'baik', 'pantesan', 'buru', 'gol', 'rkuhp', 'presiden', 'yang', 'lahir', 'curang', 'aman']
['rkuhp', 'buat', 'nyali', 'kurang']
['pasalpasal', 'rkuhp', 'lebih', 'bahaya', 'belum', 'syariahmewujudkanpersatuan']
['UU', 'omnibuslaw', 'saja', 'sahkan', 'apa', 'rkuhp']
['enggan', 'perintah', 'buka', 'draf', 'baru', 'rkuhp', 'nilai', 'tunjuk', 'bbc']
['nah', 'kamu', 'yang', 'benar', 'saja', 'sudah', 'fitnah', 'jatohnya', 'terkena', 'pasal', 'rkuhp', 'baru', 'hina', 'presiden', 'tambah', 'tindak', 'pidana', 'terorisme', 'jelas', 'dukung', 'gera', 'terorisme', 'orochi', 'm', 'top']
['maling', 'bintang', 'emon', 'vs', 'mamat', 'alkatiri', 'resiko', 'kritik', 'perintah', 'rkuhp', 'via']
['rkuhp']
['harus', 'sih', 'sadar', 'ta', 'tapi', 'ntah', 'lh', 'mungkin', 'belum', 'baca', 'rkuhp', 'yang', 'baru', 'ganggu', 'tetangga']
['salut', 'pasang', 'connect', 'zenly', 'larang', 'nongkrong', 'lawan', 'jenis', 'saling', 'tukar', 'password', 'ig',

['beberapa', 'pihak', 'kontra', 'misal', 'dewan', 'pers', 'mengritik', 'minta', 'tunda', 'kesah', 'rkuhp', 'ruu', 'kuhp', 'kesah', 'uukuhp', 'islam', 'kaffah', 'live', 'muslimah', 'bicara', 'islamsolusikehidupan', 'islamsolusikehidupan']
['pasal', 'pasal', 'yang', 'rkuhp', 'zina', 'elgebete', 'sangat', 'bahaya', 'generasi', 'masa', 'datang', 'live', 'muslimah', 'bicara', 'islamsolusikahidupan', 'kapitalismebiangmasalah']
['mnrt', 'direktur', 'ijm', 'agung', 'wisnuwardhana', 'cttn', 'penting', 'kesah', 'rkuhp', 'nilai', 'cukup', 'bahaya', 'untuk', 'konteks', 'indonesia', 'msl', 'tidak', 'brkn', 'jelas', 'clear', 'tentang', 'l', 'simak', 'sob', 'live', 'muslimah', 'bicara', 'islamsolusikehidupan', 'kapitalismebiangmasalah', 'islamkaffah']
['ada', 'debat', 'rkuhp', 'bukti', 'demokrasi', 'tak', 'mampu', 'ayom', 'orang', 'ruu', 'kuhp', 'muslimah', 'bicara', 'islamsolusikehidupan', 'kapitalismebiangmasalah']
['kesah', 'rkuhp', 'penuh', 'kontroversi', 'perintah', 'komisi', 'iii', 'dpr', 'bawa

['rkuhp', 'jadi', 'waris', 'masa', 'depan', 'bangsa']
['rkuhp', 'jadi', 'waris', 'masa', 'depan', 'bangsa']
['rkuhp', 'jadi', 'waris', 'masa', 'depan', 'bangsa']
['rkuhp', 'jadi', 'waris', 'masa', 'depan', 'bangsa']
['rkuhp', 'rupa', 'reformasi', 'sistem', 'hukum', 'pidana', 'dasar', 'nilainilai', 'pancasila', 'budaya', 'hingga', 'prinsip', 'ham']
['rkuhp', 'bawa', 'adil', 'kuhpindonesia', 'savekpk', 'hukumdankeadilan']
['sah', 'rkuhp', 'tegak', 'hukum', 'indonesia', 'kuhpindonesia', 'savekpk', 'hukumdankeadilan']
['aliansi', 'bem', 'tasikmalaya', 'duduk', 'gedung', 'dprd', 'kota', 'tasikmalaya', 'tolak', 'kesah', 'rkuhp', 'kemarin', 'des']
['kalau', 'rkuhp', 'izin', 'tidak', 'masalah']
['ancam', 'bebas', 'sipil', 'bebas', 'ekspres', 'bahas', 'terkena', 'polemik', 'UU', 'kuhp', 'sebut', 'suara', 'bebas', 'selenggara', 'diskusi', 'webinar', 'forum', 'bebas', 'angkat', 'topik', 'tantang', 'ham', 'pasca', 'kesah', 'rkuhp']
['undang', 'diskusi', 'webinar', 'forum', 'bebas', 'tantang', 'ham

['maju', 'terkena', 'mundur', 'terkena', 'rkuhp', 'rekayasa', 'putus', 'hidup', 'nista']
['usai', 'kuhp', 'baru', 'sah', 'imigrasi', 'bandara', 'soetta', 'sebut', 'jumlah', 'datang', 'wna', 'stabil', 'bhpsemarang', 'kumhampasti', 'kamipasti', 'kumhamsemakinpasti', 'kumham', 'kemenkumham', 'kemenkumhamri', 'beritaterkini', 'twitter', 'news', 'kuhp', 'rkuhp']
['maruf', 'amin', 'usah', 'marah', 'benci', 'kuhp', 'bhpsemarang', 'kumhampasti', 'kamipasti', 'kumhamsemakinpasti', 'kumham', 'kemenkumham', 'kemenkumhamri', 'beritaterkini', 'twitter', 'news', 'kuhp', 'rkuhp', 'berita']
['jaksa', 'agung', 'minta', 'jajar', 'ajar', 'pasalpasal', 'kuhp', 'baru', 'bhpsemarang', 'kumhampasti', 'kamipasti', 'kumhamsemakinpasti', 'kumham', 'kemenkumham', 'kemenkumhamri', 'beritaterkini', 'twitter', 'news', 'kuhp', 'rkuhp', 'berita']
['kesah', 'rkuhp', 'jadi', 'UU', 'cacat', 'hukum', 'langgar', 'tatib', 'dpr', 'kesah', 'terlalu', 'paksa', 'pimpin', 'negri', 'tahu', 'atur', 'erti', 'konstitusi']
['pasal',

['sosialisasi', 'diskusi', 'publik', 'terkena', 'rkuhp', 'perintah', 'laksana', 'sejak', 'tiga', 'tahun', 'lalu', 'tunda', 'rkuhp', 'kuhplebihdemokratis']
['hatihati', 'rkuhp', 'bro']
['apresiasi', 'kesah', 'rkuhp', 'jadi', 'undang', 'turut', 'kesah', 'rkuhp', 'jadi', 'kuhp', 'rupa', 'momentum', 'sejarah', 'eksistensi', 'regulasi', 'kuhp', 'nasional', 'lepas', 'ada', 'pihakpihak', 'tentu', 'berat', 'atas', 'kesah', 'kuhplebihdemokratis']
['dukung', 'rkuhp', 'buat', 'indonesia', 'savekpk', 'berantaskorupsi', 'hukumdankeadilan', 'indonesiabebaskorupsi']
['yuk', 'dukung', 'kesah', 'rkuhp', 'savekpk', 'berantaskorupsi', 'hukumdankeadilan', 'indonesiabebaskorupsi']
['sosialisasi', 'diskusi', 'publik', 'terkena', 'rkuhp', 'perintah', 'laksana', 'sejak', 'tiga', 'tahun', 'lalu', 'tunda', 'rkuhp', 'kuhplebihdemokratis']
['apresiasi', 'kesah', 'rkuhp', 'jadi', 'undang', 'turut', 'kesah', 'rkuhp', 'jadi', 'kuhp', 'rupa', 'momentum', 'sejarah', 'eksistensi', 'regulasi', 'kuhp', 'nasional', 'lepas

...

['kontroversi', 'datang', 'bbrpa', 'pasar', 'rkuhp', 'serbaserbi', 'mmc', 'islamkaffah']
['jahat', 'dpr', 'tegas', 'temu', 'demo', 'tolak', 'rkuhp']
['wah', 'pantas', 'arti', 'tolak', 'rkuhp', 'teroris', 'bismillah', 'komisaris']
['kesah', 'rkuhp', 'salah', 'satu', 'undangundang', 'produk', 'sistem', 'demokrasi', 'timbul', 'soal', 'baru', 'lengkap', 'serba', 'serbi', 'mmc', 'islamkaffah']
['kesah', 'rkuhp', 'paksa', 'menujukkan', 'hak', 'bicara', 'tipu', 'daya', 'sistem', 'kapitalis', 'benar', 'yaa', 'hak', 'bicara', 'orang', 'kaya', 'yuk', 'cek', 'serbaserbi', 'mmc', 'islamkaffah']
['penasaran', 'duit', 'buat', 'bayarin', 'orang', 'campaign', 'tuju', 'rkuhp', 'duit', 'siapa', 'sih']
['kawal', 'terus', 'rkuhp', 'jangan', 'tengelam', 'isuisu']
['kesah', 'rkuhp', 'nyata', 'dalam', 'pasalpasal', 'masalah', 'begini', 'UU', 'produk', 'sistem', 'demokrasi', 'timbul', 'soal', 'baru', 'simak', 'serba', 'serbi', 'mmc', 'islamkaffah']
['pokok', 'yang', 'hormat', 'atas', 'sana', 'jgn', 'sampai', 

['tengah', 'deras', 'arus', 'tolak', 'beberapa', 'pasal', 'rkuhp', 'dpr', 'ri', 'perintah', 'justru', 'kesah', 'jadi', 'UU', 'dlm', 'rapat', 'paripurna', 'yang', 'gelar', 'kompleks', 'parlemen', 'selasa', 'serbaserbi', 'mmc', 'islamkaffah']
['tuai', 'kontroversi', 'meski', 'banyak', 'tolak', 'msyarakat', 'dprri', 'perintah', 'tetap', 'laku', 'kesah', 'rkuhp', 'benar', 'UU', 'tak', 'bebas', 'penting', 'kuasa', 'serbaserbi', 'mmc', 'islamkaffah']
['rkuhp', 'poin', 'yang', 'cenderung', 'komunisme', 'contoh', 'nya', 'pasal', 'perzinahan', 'hina', 'lembaga', 'negara', 'kan', 'hancur', 'bebas', 'bicara', 'komunisme', 'bkn', 'cuman', 'tolak', 'nya', 'agama', 'klu', 'komunisme', 'tolak', 'nya', 'nya', 'agama', 'china', 'tidak', 'agama']
['kesah', 'rkuhp', 'ditengahtengah', 'gencar', 'tolak', 'mudah', 'atur', 'buat', 'manusia', 'diutak', 'atik', 'sangat', 'beda', 'atur', 'islam', 'kaffah', 'simak', 'serba', 'serbi', 'mmc']
['tuai', 'kontroversi', 'tengah', 'deras', 'tolak', 'masyarakat', 'kesah

['pks', 'coba', 'ambil', 'untung', 'elektoral', 'lewat', 'rkuhp']
['dapat', 'kabar', 'duduk', 'lokal', 'kata', 'gara', 'rkuhp', 'baru', 'sah', 'banyak', 'bule', 'yang', 'cancel', 'trip', 'indo']
['kemenkumham', 'lurus', 'khawatir', 'dubes', 'as', 'soal', 'rkuhp']
['semuabisakena', 'rkuhp', 'sah']
['kamu', 'kalau', 'ngomong', 'hati', 'skr', 'ingat', 'rkuhp', 'baru', 'anggap', 'hina', 'selesai', 'ente', 'ngeri', 'skr', 'mau', 'ngetik', 'tidak', 'asal']
['bom', 'bunuh', 'diri', 'heboh', 'dilainsisi', 'kasus', 'sambo', 'kelar', 'kasus', 'opm', 'jelas', 'mau', 'tangan', 'apa', 'kasus', 'kanjuruhan', 'belum', 'selesai', 'yang', 'jual', 'pulau', 'kesah', 'rkuhp', 'alih', 'isu']
['serius', 'saya', 'mau', 'nanya', 'apa', 'sih', 'produk', 'baik', 'rezim', 'kultus', 'bangun', 'moral', 'bangsa', 'yang', 'bangun', 'standart', 'saja', 'semua', 'bumbu', 'kultus', 'saja', 'rkuhp']
['kamitidaktakut', 'l', 'teroris', 'lebih', 'takut', 'rkuhp', 'masalah']
['tolak', 'rkuhp', 'jerman']
['rasa', 'usaha', 't

['tadi', 'sempat', 'lwat', 'tl', 'rkuhp', 'kalau', 'buat', 'ramai', 'usik', 'tetangga', 'tengah', 'malam', 'maybe', 'denda', 'juta', 'tapi', 'aku', 'lupa', 'akun', 'manaa']
['hilang', 'fokus', 'elu', 'yang', 'nyimpulin', 'radikal', 'nolak', 'rkuhp', 'elu', 'terus', 'yang', 'salah', 'orang', 'bukan', 'saya', 'yang', 'terlalu', 'naif', 'tapi', 'memang', 'logika', 'makhluk', 'minus', 'parah']
['rkuhp', 'bom', 'bunuh', 'diri', 'hehe']
['salah', 'satu', 'pasalpasal', 'kontroversial', 'rkhup', 'rkuhp', 'rkhup', 'kipaskipasinaja', 'kipasgeneration']
['pesan', 'adalahsiapa', 'yang', 'nolak', 'rkuhp', 'teroris']
['sah', 'nya', 'rkuhp', 'sedang', 'tuju', 'negara', 'otoriter', 'selamat', 'nikmat']
['kok', 'kesan', 'yang', 'tolak', 'rkuhp', 'teroris']
['hina', 'sama', 'memgkritikkatanya', 'pinter', 'nyata', 'bodohmembedakan', 'tidak', 'paham', 'luga', 'usah', 'sok', 'sok', 'an', 'belagu', 'kamu', 'marperbaki', 'dulu', 'etika', 'pola', 'pikir', 'kamu', 'baru', 'bahas', 'rkuhpga', 'bakal', 'nyampe',

['kalau', 'masalah', 'orang', 'mah', 'biasa', 'kan', 'mereka', 'sudah', 'stok', 'jadi', 'sudah', 'cuci', 'otak', 'siap', 'ngebom', 'kapan', 'saja', 'masalah', 'alesannya', 'aneh', 'banget', 'wkwkwk', 'rkuhp', 'kan', 'yang', 'ngeilegalin', 'pre', 'marital', 'sex', 'bukannya', 'mereka', 'hrsnya', 'neng', 'udah', 'mah', 'lokasi', 'juga', 'aneh', 'deh']
['nama', 'mere', 'yang', 'tidak', 'hadir', 'tahu', 'publik', 'jangan', 'malas', 'masuk', 'kantor', 'yang', 'terkena', 'getah', 'rkuhp', 'yang', 'kontroversi', 'kalau', 'saja', 'banyak', 'masyarakat', 'yang', 'pikir']
['bas', 'lembar', 'protes', 'kesah', 'rkuhp', 'temu', 'lokasi', 'bom', 'bunuh', 'diri', 'polsek', 'astana', 'anyar', 'kota', 'bandung', 'temu', 'dalam', 'polisi']
['kesah', 'rkuhp', 'lalu', 'sosialisasi', 'cukup', 'panjang', 'minta', 'masuk', 'ahli', 'yasonasahkanrkuhp']
['rkuhp', 'bawa', 'misi', 'dekolonialisasi', 'konsolidasi', 'harmonisasi', 'hukum', 'pidana', 'yasonasahkanrkuhp']
['perintah', 'kesah', 'rkuhp', 'jadi', 'unda

In [13]:
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open('corpus_lda.pkl', 'wb'))
dictionary.save('dictionary_lda.gensim')

In [14]:
print(len(dictionary))
print(len(corpus))

7861
5001


In [20]:
NUM_TOPICS = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=30)
ldamodel.save('modelLDA_30.gensim')

In [21]:
topics = ldamodel.show_topics(num_topics=30,num_words=250)
for topic in topics:
    print(topic)

(0, '0.071*"rkuhp" + 0.056*"sosialisasi" + 0.048*"perintah" + 0.026*"nih" + 0.026*"publik" + 0.020*"terkena" + 0.017*"pikir" + 0.017*"upaya" + 0.016*"tiga" + 0.016*"tidak" + 0.016*"tahun" + 0.015*"libat" + 0.014*"media" + 0.014*"lalu" + 0.014*"tiba" + 0.012*"laku" + 0.012*"berapa" + 0.011*"bahas" + 0.011*"laksana" + 0.011*"awal" + 0.010*"cinta" + 0.010*"ahli" + 0.010*"jokowi" + 0.009*"buru" + 0.009*"arah" + 0.009*"harga" + 0.008*"sejak" + 0.008*"point" + 0.008*"bahwa" + 0.008*"tu" + 0.008*"ramai" + 0.007*"sekali" + 0.007*"kata" + 0.007*"dorong" + 0.007*"bincang" + 0.006*"tunda" + 0.006*"tuduh" + 0.006*"rencana" + 0.005*"sebab" + 0.005*"ketua" + 0.005*"jabat" + 0.005*"suluh" + 0.005*"rusak" + 0.004*"kuhplebihdemokratis" + 0.004*"diskusi" + 0.004*"ksp" + 0.004*"meme" + 0.004*"dunia" + 0.004*"pasang" + 0.004*"sepi" + 0.004*"ibrani" + 0.004*"guna" + 0.004*"cerita" + 0.004*"fungsional" + 0.004*"mana" + 0.004*"buah" + 0.004*"bias" + 0.004*"terus" + 0.004*"partisipatif" + 0.004*"bidang" + 0.0

In [22]:
x=ldamodel.show_topics(num_topics=30, num_words=250,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Topics and Words
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

#Below Code Prints Only Words
topik=[]
for topic,words in topics_words:
     topik.append(" ".join(words))

0::['rkuhp', 'sosialisasi', 'perintah', 'nih', 'publik', 'terkena', 'pikir', 'upaya', 'tiga', 'tidak', 'tahun', 'libat', 'media', 'lalu', 'tiba', 'laku', 'berapa', 'bahas', 'laksana', 'awal', 'cinta', 'ahli', 'jokowi', 'buru', 'arah', 'harga', 'sejak', 'point', 'bahwa', 'tu', 'ramai', 'sekali', 'kata', 'dorong', 'bincang', 'tunda', 'tuduh', 'rencana', 'sebab', 'ketua', 'jabat', 'suluh', 'rusak', 'kuhplebihdemokratis', 'diskusi', 'ksp', 'meme', 'dunia', 'pasang', 'sepi', 'ibrani', 'guna', 'cerita', 'fungsional', 'mana', 'buah', 'bias', 'terus', 'partisipatif', 'bidang', 'pbhi', 'amerika', 'sengsara', 'wkwkw', 'presiden', 'artikel', 'fh', 'gara', 'gabisa', 'transparan', 'bantu', 'melulu', 'karena', 'tri', 'penjara', 'anjir', 'bnyak', 'film', 'periode', 'lewat', 'adil', 'lahir', 'promo', 'kepala', 'dana', 'ning', 'gajelas', 'bui', 'semua', 'mngesahkan', 'mmbuktikan', 'mnyatakan', 'pmbhsan', 'prtisipatif', 'trbuka', 'brmakna', 'amptdk', 'susun', 'apaapa', 'berangus', 'nemu', 'derita', 'rev

In [23]:
#convert list to string and generate
my_list=tops
# unique_string=(" ").join(my_list)
# wordcloud = WordCloud(width = 1000, height = 500).generate(unique_string)
wordcloud = WordCloud(width = 1000, height = 600, background_color="white").generate(my_list)
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig("wordcloud_all"+".png", bbox_inches='tight')
plt.show()
plt.close()

NameError: name 'tops' is not defined

In [ ]:
ldatopics = ldamodel.show_topics(formatted=False)

In [ ]:
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
def evaluate_graph(dictionary, corpus, texts, limit):
    """
    Function to display num_topics - LDA graph using c_v coherence
    
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : topic limit
    
    Returns:
    -------
    lm_list : List of LDA topic models
    c_v : Coherence values corresponding to the LDA model with respective number of topics
    """
    c_v = []
    lm_list = []
    for num_topics in range(1, limit):
        lm = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        lm_list.append(lm)
        cm = CoherenceModel(model=lm, texts=texts, dictionary=dictionary, coherence='c_v')
        c_v.append(cm.get_coherence())
        
    # Show graph
    x = range(1, limit)
    plt.plot(x, c_v)
    plt.xlabel("num_topics")
    plt.ylabel("Coherence score")
    plt.legend(("c_v"), loc='best')
    plt.show()
    
    return lm_list, c_v

In [ ]:
%%time
lmlist, c_v = evaluate_graph(dictionary=dictionary, corpus=corpus, texts=text_data, limit=30)

In [ ]:
c_v